In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast

traffic_df = pd.read_csv('도로종류별 교통량_좌표,휴게소추가_최종본.csv')
traffic_df.head()

In [ ]:
# 라이브러리
import folium
from folium import Choropleth, Circle, Marker, PolyLine
import numpy as np
import pandas as pd
import seaborn as sns
import squarify
import ast
import pickle
import warnings
import math

import geopy
from geopy.distance import geodesic

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

from mpl_toolkits.mplot3d import Axes3D 
from datetime import datetime
from haversine import haversine

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer, QuantileTransformer, RobustScaler

In [ ]:
def cal_rad(A,B):
    rad = math.atan2(A[1]-B[1], A[0]-B[0])
    return rad

def count_color(count):
    if count >=20:
        color = '#c18cd1'
    elif count >=14:
        color = '#bc65d6'
    elif count >=7:
        color = '#b940de'
    elif count >=3:
        color = '#b223de'
    else:
        color = '#9000c9'
    return color

In [ ]:
def cal_dist(x1, y1, x2, y2, a, b):
    area = abs((x1-a) * (y2-b) - (y1-b) * (x2 - a))
    AB = ((x1-x2)**2 + (y1-y2)**2) **0.5
    distance = area/AB
    return distance

In [ ]:
traffic_df['accident_count'] = 0
traffic_df.drop('Unnamed: 0',axis=1,inplace=True)
traffic_df.head()

In [ ]:
traffic_df['startpoint_coor'] = traffic_df['startpoint_coor'].apply(ast.literal_eval)
traffic_df['endpoint_coor'] = traffic_df['endpoint_coor'].apply(ast.literal_eval)

In [ ]:
acc_df = pd.read_csv('고속도로 사고 데이터.csv')
print(acc_df.shape)
acc_df.head()

In [ ]:
acc_df = acc_df[acc_df['가해운전자 차종'] == '화물']
acc_df.shape

In [ ]:
# 1.각 고속도로 사고별 최단 거리에 있는 고속도로 구간 탐색 -> 점과 직선사이의 거리 공식을 이용해 카운트
pd.options.display.float_format = '{:.5f}'.format

for i in range(len(acc_df)):
    rest_x = acc_df.iloc[i]['위도']
    rest_y = acc_df.iloc[i]['경도']
    
    min_dist_secName = ''
    min_dist = 9999999
    
    for j in range(len(traffic_df)):    
        sect_sx = traffic_df.iloc[j]['startpoint_coor'][0]
        sect_sy = traffic_df.iloc[j]['startpoint_coor'][1]
        sect_ex = traffic_df.iloc[j]['endpoint_coor'][0]
        sect_ey = traffic_df.iloc[j]['endpoint_coor'][1]
        sectName = traffic_df.iloc[j]['구간']
        
        dist = cal_dist(sect_sx,sect_sy,sect_ex,sect_ey,rest_x,rest_y)
        
        if sect_sx > sect_ex:
            tmp = sect_ex
            sect_ex = sect_sx
            sect_sx = tmp
        if sect_sy > sect_ey:
            tmp = sect_ey
            sect_ey = sect_sy
            sect_sy = tmp
        
        if min_dist > dist and ((sect_sx < rest_x and rest_x < sect_ex) or (sect_sy < rest_y and rest_y < sect_ey)):
            min_dist = dist
            min_dist_secName = sectName

    idx = traffic_df[traffic_df['구간'] == min_dist_secName].index
    traffic_df.iloc[idx,17] = traffic_df[traffic_df['구간'] == min_dist_secName]['accident_count'] + 1

In [ ]:
traffic_df['accident_count'].hist()

In [ ]:
#2. polygon.contains를 활용해서 구간별 사고건수 check
counts=[]

for i in range(len(traffic_df)):
    startpoint = traffic_df['startpoint_coor'][i]
    endpoint = traffic_df['endpoint_coor'][i]

    rad = cal_rad(startpoint, endpoint)
    deg = np.rad2deg(rad)
    dist = 1.6
    origin_start = geopy.Point(startpoint)
    origin_end = geopy.Point(endpoint)

    degree = [90+deg, 270+deg]

    poly_a = []
    poly_b = []

    for de in degree:

        destination_start = geodesic(kilometers=dist).destination(origin_start, de)
        destination_end = geodesic(kilometers=dist).destination(origin_end, de)
        new_lat1, new_lon1 = destination_start.latitude, destination_start.longitude
        new_lat2, new_lon2 = destination_end.latitude, destination_end.longitude

        poly_a.append([new_lat1, new_lon1])
        poly_b.append([new_lat2, new_lon2])
        
      
    polygon_vect = np.array([poly_a[0],poly_b[0],poly_b[1],poly_a[1]])
    polygon = Polygon(polygon_vect)

    count =0
    for i in range(len(acc_df)):
        point = Point(acc_df['위도'][i], acc_df['경도'][i])

        if polygon.contains(point):
            count+=1

    counts.append(count)